In [1]:
import glob
import pandas as pd
import numpy as np
import os
import csv
import sys

In [2]:
#Read CSV file containing the persons with an income value from spenser_ntebook_7
df_income_dir = r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\SPENSER\Data\2019\income' # use your path
df_income_file = os.path.join(df_income_dir, "df_income_all_saved_20220216.csv")
df_income = pd.read_csv(df_income_file, index_col=None, header=0)

In [3]:
len(df_income)

1283655

### Function to swap income values between people within the same occupation and same sex.

### This is required because it was observed that in general, people aged between 40-49 were earning more than expected, while people between 18-21, 22-29, 30-39, 50-59 and 60+ were earning less than expected

### The following function is just to fix first the income of people aged 18-21, just swapping their income with people aged 22-29 and 60-120

In [4]:
# fix first the income of people aged 18-21, just swapping their income with people aged 22-29 and 60-120

occupation_list = [1,2,3,4,5,6,7,8,9]

gender_list = [1,2]

age_list = [(16,17), (18,21), (22,29), (30,39), (40,49), (50,59), (60, 120)]

people_NE_income_updated_list = []



for occupation in occupation_list:
    
    print(occupation)
    
    for gender in gender_list:
        
        #Select those agents of a specific occupation and gender
        df_income_occupation_gender = df_income.loc[(df_income['Occupation'] == occupation) & (df_income['Sex'] == gender)]
        
        # Select those agents by range of age:    
        df_income_occupation_gender_18_21 = df_income_occupation_gender.loc[(df_income_occupation_gender['Age'] >= age_list[1][0]) & (df_income_occupation_gender['Age'] <= age_list[1][1])]
        df_income_occupation_gender_22_29 = df_income_occupation_gender.loc[(df_income_occupation_gender['Age'] >= age_list[2][0]) & (df_income_occupation_gender['Age'] <= age_list[2][1])]
        df_income_occupation_gender_60_120 = df_income_occupation_gender.loc[(df_income_occupation_gender['Age'] >= age_list[6][0]) & (df_income_occupation_gender['Age'] <= age_list[6][1])]


        # 10% of people between 22-29 will swap their income
        df_income_occupation_gender_22_29_change = df_income_occupation_gender_22_29.sample(int(round(len(df_income_occupation_gender_22_29)*0.069,0)))
        

        # 10% of people between 60-120 will swap their income
        df_income_occupation_gender_60_120_change = df_income_occupation_gender_60_120.sample(int(round(len(df_income_occupation_gender_60_120)*0.069,0)))
        
        
        # concatenate both dataframes
        df_income_occupation_gender_22_29_60_120_change = (pd.concat([df_income_occupation_gender_22_29_change, df_income_occupation_gender_60_120_change]))
        
        
        # people between 18-21 years selected to update their income
        df_income_occupation_gender_18_21_change = df_income_occupation_gender_18_21.sample(int(round(len(df_income_occupation_gender_22_29_60_120_change),0)))


        
        # Save into lists the income values selected in each of the previous groups:
        income_18_21_list = df_income_occupation_gender_18_21_change['Income'].tolist()
        income_22_29_60_120_list = df_income_occupation_gender_22_29_60_120_change['Income'].tolist() 

        
        #split the income_18_21_list in two equal parts
        income_22_29_list, income_60_120_list = np.split(income_18_21_list, [int(round(len(income_18_21_list) * 0.5,0))])
        
        
            
        if (len(income_22_29_60_120_list) < len(df_income_occupation_gender_18_21_change)):
                
            a = len(df_income_occupation_gender_18_21_change) - len(income_22_29_60_120_list)
            df_income_occupation_gender_18_21_change = df_income_occupation_gender_18_21_change[:-a]
                
                
                
        if (len(income_22_29_60_120_list) > len(df_income_occupation_gender_18_21_change)):
                
            aa = len(income_22_29_60_120_list) - len(df_income_occupation_gender_18_21_change)
            income_22_29_60_120_list = income_22_29_60_120_list[:-a]
                
                
                
        if (len(income_22_29_list) < len(df_income_occupation_gender_22_29_change)):
                
            b = len(df_income_occupation_gender_22_29_change) - len(income_22_29_list)
            df_income_occupation_gender_22_29_change = df_income_occupation_gender_22_29_change[:-b]
                

                
        if (len(income_22_29_list) > len(df_income_occupation_gender_22_29_change)):
                
            bb = len(income_22_29_list) - len(df_income_occupation_gender_22_29_change)
            income_22_29_list = income_22_29_list[:-bb]
                


                
        if (len(income_60_120_list) < len(df_income_occupation_gender_60_120_change)):
                
            c = len(df_income_occupation_gender_60_120_change) - len(income_60_120_list)
            df_income_occupation_gender_60_120_change = df_income_occupation_gender_60_120_change[:-c]
            

                
            
        if (len(income_60_120_list) > len(df_income_occupation_gender_60_120_change)):
                
            cc = len(income_60_120_list) - len(df_income_occupation_gender_60_120_change)
            income_60_120_list = income_60_120_list[:-cc]
            

        df_income_occupation_gender_18_21_change['Income'] = income_22_29_60_120_list
        

        df_income_occupation_gender_22_29_change['Income'] = income_22_29_list
        
   
        df_income_occupation_gender_60_120_change['Income'] = income_60_120_list


        
        # Concatenate all previous generated dataframes
        df_income_occupation_gender_updated = (pd.concat([df_income_occupation_gender_18_21_change, df_income_occupation_gender_22_29_change, df_income_occupation_gender_60_120_change]))
 
        #Append the dataframe into the temporal list
        people_NE_income_updated_list.append(df_income_occupation_gender_updated)
        
        
#concatenate all people "LONGTERM SICK" in one dataframe
df_people_NE_income_updated_all_first_stage = pd.concat(people_NE_income_updated_list, axis=0, ignore_index=True)
        
print('code has finished. Check the results!')


1
2
3
4
5
6
7
8
9
code has finished. Check the results!


In [5]:
# List containing those agents which income has been updated
agents_updated_list = df_people_NE_income_updated_all_first_stage['PID_AreaMSOA'].tolist()
        
# Remove from the original 
df_income_no_updated = df_income[~df_income['PID_AreaMSOA'].isin(agents_updated_list)]


In [6]:
len(df_people_NE_income_updated_all_first_stage)

48093

In [7]:
len(df_income_no_updated)

1235562

In [8]:
# Check if there is any dupplicates
len(df_income) - len(df_people_NE_income_updated_all_first_stage) - len(df_income_no_updated)

0

In [9]:
# Concatenate all previous generated dataframes
df_income_new = (pd.concat([df_income_no_updated, df_people_NE_income_updated_all_first_stage]))

len(df_income_new)


1283655

In [10]:
# Check again if there are duplicates
df_income_new_1 = df_income_new

# Remove duplicates
df_income_new_1 = df_income_new_1.drop_duplicates(subset='PID_AreaMSOA',keep = False)


len(df_income_new_1)


1283655

In [11]:
len(df_income)

1283655

### Check the median income value per range of age:

In [14]:
df_income_all_1 = df_income_new.loc[(df_income_new['Age'] >=16) & (df_income_new['Age'] <=17)]

df_income_all_1["Income"].median()

5500.0

In [15]:
df_income_all_2 = df_income_new.loc[(df_income_new['Age'] >=18) & (df_income_new['Age'] <=21)]

df_income_all_2["Income"].median()

9380.0

In [16]:
df_income_all_3 = df_income_new.loc[(df_income_new['Age'] >=22) & (df_income_new['Age'] <=29)]

df_income_all_3["Income"].median()

16590.0

In [17]:
df_income_all_4 = df_income_new.loc[(df_income_new['Age'] >=30) & (df_income_new['Age'] <=39)]

df_income_all_4["Income"].median()

24940.0

In [18]:
df_income_all_5 = df_income_new.loc[(df_income_new['Age'] >=40) & (df_income_new['Age'] <=49)]

df_income_all_5["Income"].median()

39880.0

In [19]:
df_income_all_6 = df_income_new.loc[(df_income_new['Age'] >=50) & (df_income_new['Age'] <=59)]

df_income_all_6["Income"].median()

26430.0

In [20]:
df_income_all_7 = df_income_new.loc[(df_income_new['Age'] >=60)]

df_income_all_7["Income"].median()

15050.0

In [21]:
df_income_new["Income"].median()

23060.0

### Function to swap income values between people within the same occupation and same sex.

### This is required because it was observed that in general, people aged between 40-49 were earning more than expected, while people between 18-21, 22-29, 30-39, 50-59 and 60+ were earning less than expected


### The following function is just to swap the income values between 22-29, 30-39, 40-49, 50-59 and 60+ in order to balance their mean gross annual income

In [64]:
## swap the income values between 22-29, 30-39, 40-49, 50-59 and 60+ in order to balance their mean gross annual income

occupation_list = [1,2,3,4,5,6,7,8,9]

gender_list = [1,2]

age_list = [(16,17), (18,21), (22,29), (30,39), (40,49), (50,59), (60, 120)]

people_NE_income_updated_list = []



for occupation in occupation_list:
    
    print(occupation)
    
    for gender in gender_list:
        
        #Select those agents of a specific occupation and gender
        df_income_occupation_gender = df_income_new.loc[(df_income_new['Occupation'] == occupation) & (df_income_new['Sex'] == gender)]
        
        # Select those agents by range of age:    
        df_income_occupation_gender_22_29 = df_income_occupation_gender.loc[(df_income_occupation_gender['Age'] >= age_list[2][0]) & (df_income_occupation_gender['Age'] <= age_list[2][1])]
        df_income_occupation_gender_30_39 = df_income_occupation_gender.loc[(df_income_occupation_gender['Age'] >= age_list[3][0]) & (df_income_occupation_gender['Age'] <= age_list[3][1])]
        df_income_occupation_gender_40_49 = df_income_occupation_gender.loc[(df_income_occupation_gender['Age'] >= age_list[4][0]) & (df_income_occupation_gender['Age'] <= age_list[4][1])]
        df_income_occupation_gender_50_59 = df_income_occupation_gender.loc[(df_income_occupation_gender['Age'] >= age_list[5][0]) & (df_income_occupation_gender['Age'] <= age_list[5][1])]
        df_income_occupation_gender_60_120 = df_income_occupation_gender.loc[(df_income_occupation_gender['Age'] >= age_list[6][0]) & (df_income_occupation_gender['Age'] <= age_list[6][1])]

        
        
        # 45% of people between 40-49 will swap their income
        df_income_occupation_gender_40_49_change = df_income_occupation_gender_40_49.sample(int(round(len(df_income_occupation_gender_40_49)*0.46,0)))
        
        # 15% of people between 50-59 will swap their income
        df_income_occupation_gender_50_59_change = df_income_occupation_gender_50_59.sample(int(round(len(df_income_occupation_gender_50_59)*0.15,0)))
        
        
        df_income_occupation_gender_40_49_50_59_change = (pd.concat([df_income_occupation_gender_40_49_change, df_income_occupation_gender_50_59_change]))
 
        
        #Split the lists of people that have to reduce their income proportionally to those people that have to increase their income
        ratio_22_29 = 0.44
        ratio_30_39 = 0.24
        ratio_60_120 = 0.32
        
        # Select those people between (22-29, 30-39 and 60-120) years that are going to swap their income with people between 40-49 years
        ## Age 22-29 --> 50% of the selected people aged 40-49
        df_income_occupation_gender_22_29_change = df_income_occupation_gender_22_29.sample(int(round(len(df_income_occupation_gender_40_49_50_59_change)* ratio_22_29,0)))
        ## Age 30-39 --> 20% of the selected people aged 40-49
        df_income_occupation_gender_30_39_change = df_income_occupation_gender_30_39.sample(int(round(len(df_income_occupation_gender_40_49_50_59_change)* ratio_30_39,0)))
        ## Age 60-120 --> 30% of the selected people aged 40-49
        df_income_occupation_gender_60_120_change = df_income_occupation_gender_60_120.sample(int(round(len(df_income_occupation_gender_40_49_50_59_change)* ratio_60_120,0)))
        
        
        
        # Save into lists the income values selected in each of the previous groups:
        #income_16_17_list = df_income_occupation_gender_16_17_change['Income'].tolist()
        income_18_21_list = df_income_occupation_gender_18_21_change['Income'].tolist()
        income_22_29_list = df_income_occupation_gender_22_29_change['Income'].tolist() 
        income_30_39_list = df_income_occupation_gender_30_39_change['Income'].tolist()
        income_40_49_list = df_income_occupation_gender_40_49_change['Income'].tolist()
        income_50_59_list = df_income_occupation_gender_50_59_change['Income'].tolist()
        income_60_120_list = df_income_occupation_gender_60_120_change['Income'].tolist()
        
        
        income_40_49_50_59_list = income_40_49_list + income_50_59_list
        
        

        income_40_49_50_59_to_22_29_list, income_40_49_50_59_to_30_39_list, income_40_49_50_59_to_60_120_list = np.split(income_40_49_50_59_list, [int(round(len(income_40_49_50_59_list) * ratio_22_29,0)), int(round(len(income_40_49_50_59_list)*(ratio_22_29 + ratio_30_39),0))])
        
        
        income_22_29_30_39_60_120__to__40_49_50_59_list = income_22_29_list + income_30_39_list + income_60_120_list
        
            
        if (len(income_40_49_50_59_to_22_29_list) < len(df_income_occupation_gender_22_29_change)):
                
            a = len(df_income_occupation_gender_22_29_change) - len(income_40_49_50_59_to_22_29_list)
            df_income_occupation_gender_22_29_change = df_income_occupation_gender_22_29_change[:-a]
                
            #print('fixed dataframe')
                
                
        if (len(income_40_49_50_59_to_22_29_list) > len(df_income_occupation_gender_22_29_change)):
                
            aa = len(income_40_49_to_22_29_list) - len(df_income_occupation_gender_22_29_change)
            income_40_49_50_59_to_22_29_list = income_40_49_50_59_to_22_29_list[:-a]
                
            #print('fixed list')
                
                
        if (len(income_40_49_50_59_to_30_39_list) < len(df_income_occupation_gender_30_39_change)):
                
            b = len(df_income_occupation_gender_30_39_change) - len(income_40_49_50_59_to_30_39_list)
            df_income_occupation_gender_30_39_change = df_income_occupation_gender_30_39_change[:-b]
                
            #print('fixed dataframe')
                
        if (len(income_40_49_50_59_to_30_39_list) > len(df_income_occupation_gender_30_39_change)):
                
            bb = len(income_40_49_50_59_to_30_39_list) - len(df_income_occupation_gender_30_39_change)
            income_40_49_50_59_to_30_39_list = income_40_49_50_59_to_30_39_list[:-bb]
                
            #print('fixed list')

                
        if (len(income_40_49_50_59_to_60_120_list) < len(df_income_occupation_gender_60_120_change)):
                
            c = len(df_income_occupation_gender_60_120_change) - len(income_40_49_50_59_to_60_120_list)
            df_income_occupation_gender_60_120_change = df_income_occupation_gender_60_120_change[:-c]
            
            #print('fixed dataframe')
                
            
        if (len(income_40_49_50_59_to_60_120_list) > len(df_income_occupation_gender_60_120_change)):
                
            cc = len(income_40_49_50_59_to_60_120_list) - len(df_income_occupation_gender_60_120_change)
            income_40_49_50_59_to_60_120_list = income_40_49_50_59_to_60_120_list[:-cc]
            
            #print('fixed list')

        
        # Swap the values

        df_income_occupation_gender_22_29_change['Income'] = income_40_49_50_59_to_22_29_list
        
        df_income_occupation_gender_30_39_change['Income'] = income_40_49_50_59_to_30_39_list
        
        df_income_occupation_gender_60_120_change['Income'] = income_40_49_50_59_to_60_120_list

        
        
        
        if (len(income_22_29_30_39_60_120__to__40_49_50_59_list) != len(df_income_occupation_gender_40_49_50_59_change)):
        
            if (len(df_income_occupation_gender_40_49_50_59_change) < len(income_22_29_30_39_60_120__to__40_49_50_59_list)):
                
                d = len(income_22_29_30_39_60_120__to__40_49_50_59_list) - len(df_income_occupation_gender_40_49_50_59_change)
                
                income_22_29_30_39_60_120__to__40_49_50_59_list = income_22_29_30_39_60_120__to__40_49_50_59_list[:-d]
                
            if (len(df_income_occupation_gender_40_49_50_59_change) > len(income_22_29_30_39_60_120__to__40_49_50_59_list)):
                
                e = len(df_income_occupation_gender_40_49_50_59_change) - len(income_22_29_30_39_60_120__to__40_49_50_59_list)
                
                df_income_occupation_gender_40_49_50_59_change = df_income_occupation_gender_40_49_50_59_change[:-e]
        

        
        df_income_occupation_gender_40_49_50_59_change['Income'] = income_22_29_30_39_60_120__to__40_49_50_59_list
        
        
        
        # Concatenate all previous generated dataframes
        df_income_occupation_gender_updated = (pd.concat([df_income_occupation_gender_22_29_change,df_income_occupation_gender_30_39_change,df_income_occupation_gender_40_49_50_59_change,df_income_occupation_gender_60_120_change]))
 
        #Append the dataframe into the temporal list
        people_NE_income_updated_list.append(df_income_occupation_gender_updated)
        
        
#concatenate all people "LONGTERM SICK" in one dataframe
df_people_NE_income_updated_all = pd.concat(people_NE_income_updated_list, axis=0, ignore_index=True)
        
print('code has finished. Check the results!')

1
2
3
4
5
6
7
8
9
code has finished. Check the results!


In [65]:
# List containing those agents which income has been updated
agents_updated_list = df_people_NE_income_updated_all['PID_AreaMSOA'].tolist()
        

# Remove from the original 
df_income_no_updated = df_income_new[~df_income_new['PID_AreaMSOA'].isin(agents_updated_list)]


In [66]:
len(df_people_NE_income_updated_all)

331076

In [67]:
len(df_income_no_updated)

952579

In [69]:
# Check if there is any duplicates or missing person
len(df_income_new) - len(df_people_NE_income_updated_all) - len(df_income_no_updated)

0

In [70]:
# Concatenate all previous generated dataframes
df_income_new_2 = (pd.concat([df_income_no_updated, df_people_NE_income_updated_all]))

len(df_income_new_2)
 

1283655

In [71]:
len(df_income_new)

1283655

In [72]:
len(df_income_new_2) - len(df_income)

0

In [73]:
len(df_income_new_2) - len(df_income_new)

0

## Check the median income value per range of age:

In [76]:
df_income_all_1 = df_income_new_2.loc[(df_income_new_2['Age'] >=16) & (df_income_new_2['Age'] <=17)]

df_income_all_1["Income"].median()

5500.0

In [77]:
df_income_all_2 = df_income_new_2.loc[(df_income_new_2['Age'] >=18) & (df_income_new_2['Age'] <=21)]

df_income_all_2["Income"].median()

9380.0

In [78]:
df_income_all_3 = df_income_new_2.loc[(df_income_new_2['Age'] >=22) & (df_income_new_2['Age'] <=29)]

df_income_all_3["Income"].median()

21140.0

In [79]:
df_income_all_4 = df_income_new_2.loc[(df_income_new_2['Age'] >=30) & (df_income_new_2['Age'] <=39)]

df_income_all_4["Income"].median()

26160.0

In [80]:
df_income_all_5 = df_income_new_2.loc[(df_income_new_2['Age'] >=40) & (df_income_new_2['Age'] <=49)]

df_income_all_5["Income"].median()

27570.0

In [81]:
df_income_all_6 = df_income_new_2.loc[(df_income_new_2['Age'] >=50) & (df_income_new_2['Age'] <=59)]

df_income_all_6["Income"].median()

24680.0

In [82]:
df_income_all_7 = df_income_new_2.loc[(df_income_new_2['Age'] >=60)]

df_income_all_7["Income"].median()

17930.0

In [85]:
df_income_new_2["Income"].median()

23060.0

In [86]:
df_income_new_2["Income"].mean()

25650.229477546538

In [88]:
#Export the df_persons_NE_household_composition_updated

df_income_new_2_20222307_correct = df_income_new_2

df_income_new_2_20222307_correct.to_csv(r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\SPENSER\Data\2019\income\df_income_new_2_20222307_correct.csv', encoding='utf-8', header=True)
